## Modelo de Poisson doble para predicción de resultados en fútbol

En este notebook se implementa el modelo de Poisson doble, que permite estimar el número de goles esperados por cada equipo en un partido en función de:

- La fuerza ofensiva y defensiva de cada equipo.
- La ventaja de jugar como local.

El objetivo principal es calcular la probabilidad de cada resultado posible (victoria local, empate o victoria visitante) y evaluar la capacidad del modelo para predecir el desenlace de partidos reales.

### Fases del análisis

1. Preparación del conjunto de datos.
2. Estimación de parámetros: ataque, defensa y localía.
3. Cálculo de goles esperados por partido.
4. Cálculo de probabilidades de resultado.
5. Validación sobre la temporada 2023-24.


In [ ]:
import pandas as pd
import numpy as np

# Cargar el dataset
df = pd.read_csv("../datasets/combined_odds_cleaned.csv")

# Asegurar tipos de datos
df['Date'] = pd.to_datetime(df['Date'])
df['Season'] = df['Season'].astype(str)

# Filtrar últimas 4 temporadas para entrenamiento
temporadas_entrenamiento = ['2019-20', '2020-21', '2021-22', '2022-23']
df_train = df[df['Season'].isin(temporadas_entrenamiento)]
df_test = df[df['Season'] == '2023-24']

print(f"Partidos de entrenamiento: {len(df_train)}")
print(f"Partidos de validación: {len(df_test)}")


## Estimación de parámetros del modelo de Poisson doble

Para cada equipo se estima:

- Una **fuerza ofensiva** (`ataque`)
- Una **fuerza defensiva** (`defensa`)
- Un parámetro común de **ventaja de jugar en casa** (`gamma`)

El modelo asume que los goles marcados por el equipo local y visitante siguen distribuciones de Poisson independientes con medias:

- λ (goles esperados del local) = exp(ataque_local - defensa_visitante + gamma)
- μ (goles esperados del visitante) = exp(ataque_visitante - defensa_local)

La estimación de parámetros se realiza mediante **máxima verosimilitud**, minimizando la log-verosimilitud negativa sobre todos los partidos del conjunto de entrenamiento.


In [ ]:
import numpy as np
from scipy.optimize import minimize
from scipy.stats import poisson
import pandas as pd

# Lista de equipos y diccionario de índices
equipos = sorted(pd.unique(df_train[['HomeTeam', 'AwayTeam']].values.ravel()))
n = len(equipos)
idx = {team: i for i, team in enumerate(equipos)}

# Función de log-verosimilitud negativa, protegida contra overflow
def logverosimilitud(params):
    ataque = params[:n]
    defensa = params[n:2*n]
    gamma = params[-1]

    log_lik = 0
    for _, row in df_train.iterrows():
        i = idx[row['HomeTeam']]
        j = idx[row['AwayTeam']]
        g_local = row['FTHG']
        g_visitante = row['FTAG']

        # Protección contra desbordamiento
        lambda_ij = np.exp(np.clip(ataque[i] - defensa[j] + gamma, -10, 10))
        mu_ji = np.exp(np.clip(ataque[j] - defensa[i], -10, 10))

        log_lik += poisson.logpmf(g_local, lambda_ij)
        log_lik += poisson.logpmf(g_visitante, mu_ji)

    return -log_lik

# Inicialización aleatoria controlada
rng = np.random.default_rng(seed=42)
x0 = np.concatenate([
    rng.normal(0, 0.1, n),  # ataque
    rng.normal(0, 0.1, n),  # defensa
    [0.1]                   # gamma
])

# Límites razonables para parámetros
bounds = [(-5, 5)] * (2*n) + [(-1, 1)]  # ataque, defensa, gamma

# Optimización con límites y sin restricciones adicionales
res = minimize(
    logverosimilitud,
    x0,
    method='L-BFGS-B',
    bounds=bounds,
    options={'maxiter': 50, 'disp': True}
)

# Extraer parámetros
ataque = dict(zip(equipos, res.x[:n]))
defensa = dict(zip(equipos, res.x[n:2*n]))
gamma = res.x[-1]

print(f"Ventaja estimada de jugar en casa (gamma): {gamma:.4f}")


## Predicción de resultados en la temporada 2023-24

Con los parámetros estimados (ataque, defensa y ventaja de localía), se calculan los goles esperados para cada partido de la temporada 2023-24:

- λ: goles esperados del equipo local
- μ: goles esperados del equipo visitante

A partir de estas medias, se obtiene la matriz de probabilidades para los resultados posibles (0–0, 0–1, ..., 5–5), asumiendo independencia entre ambos equipos.

Después, se agrupan estas probabilidades para calcular:

- Probabilidad de victoria local: \( P(goles\_local > goles\_visitante) \)
- Probabilidad de empate: \( P(goles\_local = goles\_visitante) \)
- Probabilidad de victoria visitante: \( P(goles\_local < goles\_visitante) \)

Finalmente, se compara el resultado más probable con el resultado real (`FTR`) para evaluar la precisión del modelo.


In [ ]:
from scipy.stats import poisson

# Máximo número de goles considerados (0 a 5)
max_goals = 6

# Lista para guardar predicciones
predicciones = []

for _, row in df_test.iterrows():
    equipo_local = row['HomeTeam']
    equipo_visitante = row['AwayTeam']

    if equipo_local not in ataque or equipo_visitante not in ataque:
        continue

    # Calcular goles esperados con control de estabilidad
    lambda_home = np.exp(np.clip(ataque[equipo_local] - defensa[equipo_visitante] + gamma, -10, 10))
    mu_away = np.exp(np.clip(ataque[equipo_visitante] - defensa[equipo_local], -10, 10))

    # Matriz conjunta de probabilidades
    prob_home = poisson.pmf(range(max_goals), lambda_home)
    prob_away = poisson.pmf(range(max_goals), mu_away)
    matriz = np.outer(prob_home, prob_away)

    # Sumar probabilidades por tipo de resultado
    p_home_win = np.tril(matriz, -1).sum()
    p_draw = np.trace(matriz)
    p_away_win = np.triu(matriz, 1).sum()

    # Resultado más probable según el modelo
    pred_result = np.argmax([p_home_win, p_draw, p_away_win])
    pred_label = ['H', 'D', 'A'][pred_result]

    # Guardar predicción
    predicciones.append({
        'HomeTeam': equipo_local,
        'AwayTeam': equipo_visitante,
        'FTR_real': row['FTR'],
        'P_H': p_home_win,
        'P_D': p_draw,
        'P_A': p_away_win,
        'Pred': pred_label
    })

# Convertir a DataFrame
df_preds = pd.DataFrame(predicciones)

# Evaluar precisión de clasificación
accuracy = (df_preds['Pred'] == df_preds['FTR_real']).mean()
print(f"Precisión del modelo (aciertos exactos en 2023-24): {accuracy:.3f}")

## Evaluación del modelo: Matriz de confusión

Para analizar el rendimiento del modelo de Poisson doble al predecir el resultado del partido (H, D, A), se utiliza una matriz de confusión.

Esta matriz compara:

- Las predicciones generadas por el modelo (columna "Pred").
- Los resultados reales observados en los datos (columna "FTR_real").

Permite identificar:
- Cuántas veces el modelo acierta exactamente.
- Dónde tiende a equivocarse (por ejemplo, si sobreestima victorias locales o subestima empates).

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

# Calcular matriz de confusión
etiquetas = ['H', 'D', 'A']
cm = confusion_matrix(df_preds['FTR_real'], df_preds['Pred'], labels=etiquetas)

# Mostrar como gráfico
fig, ax = plt.subplots(figsize=(6, 5))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=etiquetas, yticklabels=etiquetas)
plt.xlabel("Predicción del modelo")
plt.ylabel("Resultado real")
plt.title("Matriz de confusión - Resultados 2023-24")
plt.show()
